<a href="https://colab.research.google.com/github/dbwofla11/DeepLearning_Training/blob/master/ANN/FFNN%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1일차 ✈  단일 입력 FFNN 구현하기
* TensorFlow없이 구현하기
* TensorFlow있이 구현하기

# 1. 단일 은닉층 FFNN ( AND 문제 )


In [ ]:
# 단일 은닉층 모델 AND 문제
import numpy as np

def sigmoide(x : float):
  return 1 / (1 + np.exp(-x))

def derivada_sigmoide(x : float):
  return x * (1 - x)

class FFNN:
  def __init__(self , input_Size , hidden_Size , output_Size):
    self.input_Size = input_Size
    self.hidden_Size = hidden_Size
    self.output_Size = output_Size

    # 가중치 초기화
    self.W1 = np.random.randn(self.input_Size , self.hidden_Size) # I -> H
    self.W2 = np.random.randn(self.hidden_Size , self.output_Size) # H -> O

    self.b1 = np.zeros((1 , self.hidden_Size)) # H bias
    self.b2 = np.zeros((1 , self.output_Size)) # O bias

# 포워드와 벡워드만 잘 수식보면서 이해하면 됨 ( 수식대로만 친거임 )
  def forward(self , X):
      self.z1 = np.dot(X , self.W1) + self.b1
      self.a1 = sigmoide(self.z1)

      self.z2 = np.dot(self.a1 , self.W2) + self.b2
      self.a2 = sigmoide(self.z2)

      return self.a2

  def backward(self , X , Y , learning_rate):
      m = X.shape[0] # 샘플 수

      # 출력층 오차
      dZ2 = self.a2 - Y # 예측 - 실제
      dW2 = (1 / m) * np.dot(self.a1.T , dZ2)
      db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)

      # 은닉층 오차
      dA1 = np.dot(dZ2 , self.W2.T)
      dZ1 = dA1 * derivada_sigmoide(self.a1)
      dW1 = (1 / m) * np.dot(X.T , dZ1)
      db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)

      # 가중치와 편향 업데이트
      self.W1 -= learning_rate * dW1
      self.b1 -= learning_rate * db1
      self.W2 -= learning_rate * dW2
      self.b2 -= learning_rate * db2


  def train(self, X , Y , epochs , learning_rate):
      for epoch in range(epochs):
        self.forward(X)
        self.backward(X , Y , learning_rate)


        if epoch % 100 == 0:
          loss = np.mean(np.square(Y - self.a2))  # MSE (Mean Squared Error) 실제와 예측
          print(f"Epoch {epoch}, Loss: {loss}")


  def predict(self , X):
      return self.forward(X)

# 예시 데이터
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # 입력 데이터 (AND 문제)
Y = np.array([[0], [1], [1], [0]])  # 출력 데이터 (AND 문제에 대한 결과)

# 네트워크 인스턴스 생성 (2개 입력, 4개 은닉층 뉴런, 1개 출력)
input_size = 2
hidden_size = 4
output_size = 1
nn = FFNN(input_size, hidden_size, output_size)

# 모델 학습
nn.train(X, Y, epochs=10000, learning_rate=0.1)

# 예측
predictions = nn.predict(X)
print("\nPredictions:")
print(predictions)

Epoch 0, Loss: 0.2524195337104239
Epoch 100, Loss: 0.25104141770850463
Epoch 200, Loss: 0.24990062091141851
Epoch 300, Loss: 0.2488181372960737
Epoch 400, Loss: 0.2476699969208191
Epoch 500, Loss: 0.24633939817688333
Epoch 600, Loss: 0.24470439389189913
Epoch 700, Loss: 0.2426358738623121
Epoch 800, Loss: 0.24000234616667976
Epoch 900, Loss: 0.23667591137143057
Epoch 1000, Loss: 0.23253531622210533
Epoch 1100, Loss: 0.2274703119299119
Epoch 1200, Loss: 0.22139943971359757
Epoch 1300, Loss: 0.21430965539100438
Epoch 1400, Loss: 0.20630754155913214
Epoch 1500, Loss: 0.19764760816717702
Epoch 1600, Loss: 0.18869810313455887
Epoch 1700, Loss: 0.17984068834560382
Epoch 1800, Loss: 0.17135241105248455
Epoch 1900, Loss: 0.16332833072415504
Epoch 2000, Loss: 0.15566323481905953
Epoch 2100, Loss: 0.14807315330677967
Epoch 2200, Loss: 0.14013636418981104
Epoch 2300, Loss: 0.13135815302100529
Epoch 2400, Loss: 0.121283166301457
Epoch 2500, Loss: 0.10966374782395863
Epoch 2600, Loss: 0.09664113442